## Version check

우리는 scikit learn 에서 제공하는 20NewsGroups data 를 이용하여 document classification 을 하는 multi-layer feed-foward neural network 를 만들어봅니다.

현재 실습의 torch 버전은 1.0.1 입니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

print(torch.__version__)

1.4.0+cpu


## Load data

20NewsGroups 의 모든 데이터를 이용해도 되지만, 빠른 확인을 위하여 네 개의 카테고리만 이용합니다. 20NewsGroups 은 20 개의 카테고리로 분류된 뉴스 문서 집합입니다.

In [2]:
# Load training set and test set
categories = [
    'rec.sport.baseball',
    'soc.religion.christian',
    'comp.windows.x',
    'sci.space'
]

removals = ('headers', 'footers', 'quotes')

remove 에 'headers', 'footers', 'quotes' 를 넣으면 뉴스의 header 들이 제거 된 text 만 받을 수 있습니다.

In [3]:
newsgroups_train = fetch_20newsgroups(
    subset='train', remove=removals, categories=categories)

newsgroups_test = fetch_20newsgroups(
    subset='test', remove=removals, categories=categories)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


여기서 text 와 category label 을 분리합니다.

In [4]:
data_train = newsgroups_train.data
y_train = newsgroups_train.target
data_test = newsgroups_test.data
y_test = newsgroups_test.target

## Vectorizing

TF-IDF vectorizer 를 이용하여 vectorizing 을 합니다. 이 때 학습 데이터 기준에서 word index 가 학습되도록 학습데이터에는 fit_transform 을 적용하고, 테스트 데이터에는 transform 을 적용합니다. 학습 데이터에 존재하지 않는 단어는 테스트 단어에서 벡터화 되지 않습니다.

In [5]:
vectorizer = TfidfVectorizer(min_df=5)
x_train = vectorizer.fit_transform(data_train)
x_test = vectorizer.transform(data_test)

print('train x = {}, y = {}'.format(x_train.shape, y_train.shape))
print('test  x = {}, y = {}'.format(x_test.shape, y_test.shape))

train x = (2382, 6535), y = (2382,)
test  x = (1584, 6535), y = (1584,)


## DataSet and DataLoader

DataSet 은 DataLoader 의 input 입니다. 각 index 에 해당하는 값을 return 하도록 getitem 과 데이터셋의 크기를 측정할 수 있는 len 만 구현하면 됩니다. 만약 데이터가 너무 큰 경우에는 init 에 파일 주소를 입력받고, getitem 에서 파일을 하나씩 읽어도 됩니다.

In [6]:
class NewsGroupDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        x = torch.FloatTensor(self.X[i].todense())[0]
        y = torch.LongTensor(self.y[i:i+1])
        return x, y

train_set = NewsGroupDataset(x_train, y_train)
test_set = NewsGroupDataset(x_test, y_test)

print('len trainset = {}'.format(len(train_set)))
print('len testset = {}'.format(len(test_set)))
train_set[0]

len trainset = 2382
len testset = 1584


(tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([0]))

DataLoader 는 Dataset 을 이용하여 minibatch 를 만들어주고 shuffling 도 지원해주는 유틸입니다.

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64)

In [8]:
for x_batch, y_batch in train_loader:
    print(x_batch.size())
    print(y_batch.size())
    break

torch.Size([16, 6535])
torch.Size([16, 1])


학습데이터는 2382 개였지만, batch size 가 16 이기 때문에 mini batch 의 개수는 149 개 입니다.

In [9]:
len(train_loader)

149

## Model

Feed forward neural netowrk 를 만듭니다. torch.nn.Module 을 상속하여 init, forward 함수만 구현하면 됩니다.

init 을 구현할 때에는 Python 의 상속처럼 super().\_\_init\_\_() 을 실행해야 합니다. 우리는 3 개의 hidden layer 로 이뤄진 feed forward 를 만들 것입니다. 각각은 (input, hidden 1), (hidden 1, hidden 2), (hidden 2, classes) 의 크기로 이뤄진 weight matrix 를 지닙니다. bias 역시 학습하도록 설정합니다.

```python
self.fc_1 = nn.Linear(in_features = input_dim, out_features = hidden_1_dim, bias=True)
self.fc_2 = nn.Linear(in_features = hidden_1_dim, out_features = hidden_2_dim, bias=True)
self.fc_3 = nn.Linear(in_features = hidden_2_dim, out_features = n_classes, bias = True)
```

우리는 Linear layer 만을 만들었을 뿐, activation function 은 아직 만들지 않았습니다. hidden layer 1, 2 의 output 에 대하여 ReLU 를 적용합니다.

```python
def forward(self, x):
    out = F.relu(self.fc_1(x))
    out = F.relu(self.fc_2(out))
    return self.fc_3(out)
```

In [10]:
class FeedForwardNN(nn.Module):
    def __init__ (self, input_dim, hidden_1_dim, hidden_2_dim, n_classes):
        super(FeedForwardNN, self).__init__()
        self.fc_1 = nn.Linear(input_dim, hidden_1_dim, bias=True)
        self.fc_2 = nn.Linear(hidden_1_dim, hidden_2_dim, bias=True)
        self.fc_3 = nn.Linear(hidden_2_dim, n_classes, bias = True)

    def forward(self, x):
        out = F.relu(self.fc_1(x))
        out = F.relu(self.fc_2(out))
        return self.fc_3(out)

## Parameters

numpy.unique 는 numpy.ndarray 의 값의 set 입니다. 네 개의 카테고리가 각각 0, 1, 2, 3 으로 encoding 되어 있습니다.

In [11]:
np.unique(y_train)

array([0, 1, 2, 3])

학습에 이용할 패러매터를 정의합니다. epochs, mini batch size, hidden 1, hidden 2 의 크기를 정의합니다.

우리가 정의한 네트워크의 구조는 아래와 같습니다. 6,535 개의 단어로 표현된 문서가 128, 32 차원을 거쳐 4 개의 클래스로 분류됩니다.

    6535 - 128 - 32 - 4


In [14]:
# Parameters
epochs = 10

input_dim = x_train.shape[1]
hidden_1_dim = 128
hidden_2_dim = 32
n_classes = np.unique(y_train).shape[0]
n_data = x_train.shape[0]

print('layer = {} - {} - {} - {}'.format(
    input_dim, hidden_1_dim, hidden_2_dim, n_classes))
print('n data = {}'.format(n_data))

layer = 6535 - 128 - 32 - 4
n data = 2382


## Model, loss function, optimizer

실제로 모델을 만듭니다. loss function (criterion) 과 optimizer 는 regression 과 비슷합니다. 단, classification 이기 때문에 Cross Entropy loss 를 이용합니다.

In [15]:
# Model
model = FeedForwardNN(
    input_dim,
    hidden_1_dim,
    hidden_2_dim,
    n_classes
)

# Parameter for the optimizer
learning_rate = 0.001

# Loss and optimizer
loss_func = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate
)

정의한 바와 같이 6535 - 128 - 32 구조의 hidden layer 를 지닌, 4 개의 클래스를 분류하는 neural network 가 만들어졌습니다.

In [16]:
print(model)

FeedForwardNN(
  (fc_1): Linear(in_features=6535, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=32, bias=True)
  (fc_3): Linear(in_features=32, out_features=4, bias=True)
)


## Train function

epochs 만큼 iteration 을 돕니다. 각 epoch 마다의 누적 loss 를 저장하기 위한 준비를 합니다.

```python
for epoch in range(epochs):
    loss_sum = 0
    # TODO
```

항상 optimizer 에 이전 step 에서 이용한 gradient 를 지운 뒤, forwarding, back-propagation 을 합니다.

```python
for i in range(n_data // batch_size):
    # ...
    optimizer.zero_grad()
```

prediction, loss 계산, 그리고 back propagation 을 수행합니다.

```python
for i in range(n_data // batch_size):
    # ...
    y_pred = model(x_batch)
    loss = criterion(y_pred, y_batch)
    loss.backward()
    optimizer.step()
```

loss 의 data 에는 loss 값이 저장되어 있습니다. 이는 torch.Tensor 이니 numpy() 를 이용하여 숫자로 변환하여 loss_sum 에 누적합니다. 매 epoch 마다 loss 가 줄어듦을 확인할 수 있습니다.

In [17]:
def train(loader, model, loss_func, optimizer, epochs):

    for epoch in range(epochs):
        loss_sum = 0
        n_batches = len(loader)

        for i, (x_batch, y_batch) in enumerate(loader):
            optimizer.zero_grad()
            y_pred = model(x_batch)
            # size: (batch, 1) -> (batch,)
            y_batch = y_batch.squeeze(dim=1)
            loss = loss_func(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            loss_sum += loss.data.numpy()
            if i % 10 == 0:
                loss_tmp = loss_sum / (i+1)
                print(f'\repoch = {epoch}, batch = {i+1}/{n_batches}, training loss = {loss_tmp:.4}', end='')
        print(f'\repoch = {epoch}/{epochs} training loss = {loss_tmp:.4}')

    return model

model = train(train_loader, model, loss_func, optimizer, epochs)

epoch = 0/10 training loss = 0.9946g loss = 0.9946
epoch = 1/10 training loss = 0.1481g loss = 0.1481
epoch = 2/10 training loss = 0.05485 loss = 0.05485
epoch = 3/10 training loss = 0.03889 loss = 0.03889
epoch = 4/10 training loss = 0.03313 loss = 0.03313
epoch = 5/10 training loss = 0.03105 loss = 0.03105
epoch = 6/10 training loss = 0.02961 loss = 0.02961
epoch = 7/10 training loss = 0.03051 loss = 0.03051
epoch = 8/10 training loss = 0.0299g loss = 0.02992
epoch = 9/10 training loss = 0.02974 loss = 0.02974


## Test accuracy

테스트 성능도 DataLoader 를 이용할 수 있습니다.

In [18]:
for x_batch, y_batch in test_loader:
    y_batch = y_batch.squeeze(dim=1)
    y_pred = model(x_batch)
    score, indices = torch.max(y_pred.data, dim=1)    
    break

y_pred 의 size 는 (batch, class) 입니다. torch.max 는 torch.Tensor 에 대하여 dim 기준으로 최대값과 그 값을 지니는 index 를 출력합니다. (batch, class) size 에서 class column 기준으로 max 값을 찾습니다. return 은 max value 와 index 로 나뉘어져 됩니다.

In [19]:
print(y_batch.size())
print(y_pred.size())
print(score.size(), indices.size())

torch.Size([64])
torch.Size([64, 4])
torch.Size([64]) torch.Size([64])


In [20]:
print(score[:3])
print(indices[:3])

tensor([ 2.0251, 11.4719,  9.8125])
tensor([3, 3, 2])


그런데 dropout 과 같은 기법은 학습 시에는 작동하고, 테스트 시에는 작동하지 않습니다. 그렇기 때문에 테스트 시에는 nn.Module 에 eval() 함수를, 학습 시에는  train() 함수를 실행하여 각각의 모드가 실행될 수 있도록 합니다. eval() 이 한 번 실행되면 모델은 더 학습되지 않습니다. 추가 학습을 하려면 반드시 train() 을 실행시켜야 합니다.

In [22]:
def test_accuracy(model, test_loader):
    n_corrects, n_total = 0, 0
    for x_batch, y_batch in test_loader:
        y_batch = y_batch.squeeze(dim=1)
        y_pred = model(x_batch)
        score, indices = torch.max(y_pred.data, dim=1)
        n_corrects += (indices == y_batch).sum().numpy()
        n_total += y_pred.size()[0]
    accuracy = n_corrects / n_total
    return accuracy

model.eval()
accuracy = test_accuracy(model, test_loader)
print(f'test accuracy = {accuracy:.4}')

test accuracy = 0.911


## IO

학습된 모델은 save 를 이용하여 저장하고 load 를 이용하여 불러올 수 있습니다.

In [23]:
model_path = 'newsgroup_net.pt'
torch.save(model, model_path)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type FeedForwardNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [25]:
loaded_model = torch.load(model_path)
loaded_model.eval()
test_accuracy(loaded_model, test_loader)

0.9109848484848485